In [23]:
%matplotlib inline
import pandas as pd
from pandas import set_option
#Limit dataframe exibition to 5 rows
set_option("display.max_rows", 10)
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gc
from sklearn import datasets, linear_model

In [24]:
df = pd.read_csv('./processed/prev_com_cd_ibge.csv', encoding='latin1')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [25]:
ibge = df['cd_ibge'].unique()

In [26]:
df2 = df

In [27]:
municipios = pd.DataFrame(df.groupby(by=['cd_ibge','estado','municipio']).first()).reset_index()

municipios.drop(['ano','vl_emissao','vl_arrecadado'], axis=1, inplace=True)

In [28]:
df2['dif'] = df2['vl_arrecadado'] - df2['vl_emissao']

Vamos predizer as emissões utilizando regressão linear

In [31]:
predictions_dif = {}
for codigo in ibge:
    regr = linear_model.LinearRegression(n_jobs=-1)
    X = df2.loc[df['cd_ibge']==codigo, ['ano']]
    y = df2.loc[df['cd_ibge']==codigo, ['dif']]
    regr.fit(X,y)    
    predictions_dif[codigo] = regr.predict(2017)[0][0]

In [32]:
df_dif = pd.DataFrame(columns = ['cd_ibge','dif'])
df_dif['cd_ibge'] = predictions_dif.keys()
df_dif['dif'] = predictions_dif.values()

In [33]:
predicoes_dif = pd.merge(df_dif, municipios, how='inner', on='cd_ibge')

In [34]:
predicoes_dif.sort_values(by='dif', ascending=False).head(10)

,cd_ibge,dif,estado,municipio
3865,3550308,4.496491e+10,São Paulo,São Paulo
2626,5300108,1.093944e+10,Distrito Federal,Brasília
3866,3304557,9.408826e+09,Rio de Janeiro,Rio de Janeiro
2812,3106200,3.717744e+09,Minas Gerais,Belo Horizonte
3690,4106902,3.200126e+09,Paraná,Curitiba
1813,4314902,2.604160e+09,Rio Grande do Sul,Porto Alegre
990,3509502,1.703438e+09,São Paulo,Campinas
2167,3534401,1.262103e+09,São Paulo,Osasco
5415,3505708,1.190075e+09,São Paulo,Barueri
3508,3205309,7.256860e+08,Espírito Santo,Vitória


In [35]:
predicoes_dif.sort_values(by='dif', ascending=True).head(10)

,cd_ibge,dif,estado,municipio
2906,3547809,-1.680109e+09,São Paulo,Santo André
4995,3301702,-1.445198e+09,Rio de Janeiro,Duque de Caxias
3988,3304904,-1.430942e+09,Rio de Janeiro,São Gonçalo
3110,3548708,-1.308027e+09,São Paulo,São Bernardo do Campo
3164,3548500,-1.178660e+09,São Paulo,Santos
2341,3136702,-1.172485e+09,Minas Gerais,Juiz de Fora
3749,2927408,-1.112247e+09,Bahia,Salvador
287,2704302,-9.917287e+08,Alagoas,Maceió
3384,3303302,-9.640182e+08,Rio de Janeiro,Niterói
4470,3306305,-9.258704e+08,Rio de Janeiro,Volta Redonda


In [36]:
predicoes_dif['dif'].sum()

-131912998237.25685

In [43]:
regioes = pd.read_csv('./src/regioes-br.csv', encoding='latin1', sep=';')

In [46]:
prd = pd.merge(predicoes_dif, regioes, how='inner', on='estado')

In [55]:
pd.DataFrame(prd.groupby(by='regiao')['dif'].sum()).reset_index().sort_values(by='dif', ascending=False)

,regiao,dif
0,CENTRO OESTE,3.280542e+09
2,NORTE,-9.474878e+09
3,SUDESTE,-2.671430e+10
4,SUL,-3.095914e+10
1,NORDESTE,-6.804522e+10


In [37]:
predicoes_dif.to_csv('./processed/predicoes_2017.csv', encoding='latin1')

In [59]:
predictions_emissoes = {}
for codigo in ibge:
    regr = linear_model.LinearRegression(n_jobs=-1)
    X = df.loc[df['cd_ibge']==codigo, ['ano']]
    y = df.loc[df['cd_ibge']==codigo, ['vl_emissao']]
    regr.fit(X,y)    
    predictions_emissoes[codigo] = regr.predict(2017)[0][0]

Agora vamos  predizer as arrecadações

In [60]:
predictions_arrecadado = {}
for codigo in ibge:
    regr = linear_model.LinearRegression(n_jobs=-1)
    X = df.loc[df['cd_ibge']==codigo, ['ano']]
    y = df.loc[df['cd_ibge']==codigo, ['vl_arrecadado']]
    regr.fit(X,y)    
    predictions_arrecadado[codigo] = regr.predict(2017)[0][0]

Agora vamos montar dataframes para facilitar a manipulação dos dados

In [95]:
df_arrec = pd.DataFrame(columns = ['cd_ibge','vl_arrecadado'])
df_arrec['cd_ibge'] = predictions_arrecadado.keys()
df_arrec['vl_arrecadado'] = predictions_arrecadado.values()

In [96]:
df_emis = pd.DataFrame(columns = ['cd_ibge','vl_emissao'])
df_emis['cd_ibge'] = predictions_emissoes.keys()
df_emis['vl_emissao'] = predictions_emissoes.values()

Juntando os 2 dataframes

In [97]:
predicoes = pd.merge(df_arrec, df_emis, how='inner', on='cd_ibge')

Adicionando informações de estado e cidade

In [134]:
predicoes = pd.merge(predicoes, municipios, how='inner', on='cd_ibge')

In [137]:
predicoes['dif'] = predicoes['vl_arrecadado'] - predicoes['vl_emissao']

In [138]:
predicoes['indice'] = predicoes['vl_arrecadado'] / predicoes['vl_emissao']

In [58]:
predicoes

NameError: name 'predicoes' is not defined

In [141]:
df.loc[df['cd_ibge']==3506201]

,municipio,estado,ano,vl_emissao,cd_ibge,vl_arrecadado
3336,Bento de Abreu,São Paulo,2000,783918,3506201,413520
8906,Bento de Abreu,São Paulo,2001,883695,3506201,392017
...,...,...,...,...,...,...
86886,Bento de Abreu,São Paulo,2015,4733895,3506201,0
92456,Bento de Abreu,São Paulo,2016,5392249,3506201,0
